In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [3]:
vars_df = pd.read_pickle('female_subject_direction_ICA_deltas_nIDP_STEVEnorm_deconf.pkl')

with open('varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
vars_categories = np.loadtxt("vars_categories.txt", dtype=str, delimiter='\n')

In [4]:
vars_df

,modality,bonf,fdr,dataframe
0,IC = 0,5.509794,3.524662,idx ...
1,IC = 1,5.509794,3.201534,idx ...


In [5]:
vars_df.iloc[0].dataframe

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
0,0,Ethnic background (0.0),Ethnic Background,0.027647,1.957215,0.627484,0.050378,0.027647,1.297762,0.202398
1,1,Ethnic background (1.0),Ethnic Background,0.035055,1.014186,0.919212,0.310787,0.035055,0.507537,0.036584
2,2,Ethnic background (2.0),Ethnic Background,0.039270,1.482522,0.793821,0.138423,0.039270,0.858792,0.100277
3,3,Genotype measurement batch (0.0),Genetic Markers,0.049738,3.478130,0.073135,0.000509,0.049738,3.292996,1.135872
4,4,Heterozygosity (0.0),Genetic Markers,0.009976,0.696801,0.984174,0.485960,0.009976,0.313399,0.006928
...,...,...,...,...,...,...,...,...,...,...
16167,17519,Methods of self-harm used (0.2),Mental Health,-0.087413,0.690939,0.984649,0.492183,0.087413,0.307874,0.006719
16168,17521,Actions taken following self-harm (0.1),Mental Health,0.201304,2.378963,0.453312,0.018772,0.201304,1.726492,0.343603
16169,17522,Actions taken following self-harm (0.2),Mental Health,0.360300,3.231528,0.155751,0.001878,0.360300,2.726298,0.807569
16170,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.025426,1.606569,0.747523,0.108228,0.025426,0.965660,0.126375


In [6]:
current_mods = vars_df.modality.values
modality_names = [
    'Subject-Direction ICA Component 1',
    'Subject-Direction ICA Component 2'
]

for idx, mod in enumerate(current_mods):
    vars_df.loc[vars_df["modality"] == mod, "modality"] = modality_names[idx]
    

In [7]:
# idxs = [1, 2, 14, 15]
# idxs = [3,4,5,6,7,8,9,10,11,12,13]

ordered_cols = [
     'Modality',
     'bonf',
     'passes_bonf',
     'fdr',
     'passes_fdr',
     'idx',
     'names',
     'Categories',
     'pearson_r',
     'abs_pearson_r',
     't_test_statistic',
     'p_values',
     'p_values_corrected',
     'log_p_values',
     'log_p_values_corrected', 
]

# for idx in idxs:
#     if idx == idxs[0]:
for idx in range(len(vars_df)):
    if idx == 0:
        df = vars_df.iloc[idx].dataframe
        df['Modality'] = vars_df.iloc[idx].modality
        df['bonf'] = vars_df.iloc[idx].bonf
        df['fdr'] = vars_df.iloc[idx].fdr
        df['passes_bonf'] = df.log_p_values > df.bonf
        df['passes_fdr'] = df.log_p_values > df.fdr
        df = df[ordered_cols]
        df = df.drop(columns=['abs_pearson_r'])
    else:
        df2 = vars_df.iloc[idx].dataframe
        df2['Modality'] = vars_df.iloc[idx].modality
        df2['bonf'] = vars_df.iloc[idx].bonf
        df2['fdr'] = vars_df.iloc[idx].fdr
        df2['passes_bonf'] = df2.log_p_values > df2.bonf
        df2['passes_fdr'] = df2.log_p_values > df2.fdr
        df2 = df2[ordered_cols]
        df2 = df2.drop(columns=['abs_pearson_r'])
        df = pd.concat([df, df2])
        del df2

In [8]:
# categories = sorted(list(set(df.Categories.to_list())))

# first_df = True

# for cat in categories:
#     df_tmp = df[(df.Categories==cat) & (df.passes_bonf==True)].sort_values('log_p_values', ascending=False)
#     df_tmp = df_tmp.head(10)
#     if len(df_tmp) == 0:
#         continue
#     if first_df == True:
#         df_print = df_tmp
#         first_df = False
#     else:
#         df_print = pd.concat([df_print, df_tmp])
        
# df_print = df_print.drop(columns=[
#     'bonf', 'passes_bonf', 'fdr', 'passes_fdr', 'idx', 't_test_statistic', 'p_values',
#     'p_values_corrected', 'log_p_values_corrected'
# ])

# df_print_colums_order = [
#     'Modality', 'log_p_values', 'pearson_r', 'Categories', 'names'
# ]

# df_print = df_print[df_print_colums_order]

# df_print = df_print.round(decimals=3)

# df_print = df_print.rename(
#     columns={
#         'log_p_values' : '-log(p)',
#         'pearson_r' : 'Correlation (r)',
#         'Categories': 'Variable Category',
#         'names' : 'Variable Description'
#     }
# )

# df_print



In [9]:
# df_print.to_csv('Famale_top_nIDPs.csv', index=False)

In [10]:
df

,Modality,bonf,passes_bonf,fdr,passes_fdr,idx,names,Categories,pearson_r,t_test_statistic,p_values,p_values_corrected,log_p_values,log_p_values_corrected
0,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,0,Ethnic background (0.0),Ethnic Background,0.027647,1.957215,0.050378,0.627484,1.297762,0.202398
1,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,1,Ethnic background (1.0),Ethnic Background,0.035055,1.014186,0.310787,0.919212,0.507537,0.036584
2,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,2,Ethnic background (2.0),Ethnic Background,0.039270,1.482522,0.138423,0.793821,0.858792,0.100277
3,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,3,Genotype measurement batch (0.0),Genetic Markers,0.049738,3.478130,0.000509,0.073135,3.292996,1.135872
4,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,4,Heterozygosity (0.0),Genetic Markers,0.009976,0.696801,0.485960,0.984174,0.313399,0.006928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16167,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17519,Methods of self-harm used (0.2),Mental Health,-0.182805,1.464076,0.148225,0.818487,0.829078,0.086988
16168,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17521,Actions taken following self-harm (0.1),Mental Health,-0.020585,0.238343,0.811979,1.000000,0.090455,-0.000000
16169,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17522,Actions taken following self-harm (0.2),Mental Health,0.146455,1.238687,0.219599,0.880943,0.658370,0.055052
16170,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.023521,1.486123,0.137326,0.802019,0.862249,0.095815


In [11]:
df = df.rename(
    columns={
        'log_p_values' : '-log(P)',
        'pearson_r' : 'Correlation (r)',
        'Categories': 'Variable Category',
        'names' : 'Variable Description',
        'bonf': 'Bonferroni Threshold',
        'passes_bonf': 'Passes Bonf.',
        'fdr': 'False Discovery Rate (FDR)',
        'passes_fdr': 'Passes FDR',
        'idx': 'Caterogy ID',
        't_test_statistic': 'T-Values',
        'p_values': 'P-Values',
        'p_values_corrected': 'FDR Adjusted P-Value',
        'log_p_values_corrected': 'FDR Adjusted -log(P)'
    }
)

df

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
0,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,0,Ethnic background (0.0),Ethnic Background,0.027647,1.957215,0.050378,0.627484,1.297762,0.202398
1,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,1,Ethnic background (1.0),Ethnic Background,0.035055,1.014186,0.310787,0.919212,0.507537,0.036584
2,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,2,Ethnic background (2.0),Ethnic Background,0.039270,1.482522,0.138423,0.793821,0.858792,0.100277
3,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,3,Genotype measurement batch (0.0),Genetic Markers,0.049738,3.478130,0.000509,0.073135,3.292996,1.135872
4,Subject-Direction ICA Component 1,5.509794,False,3.524662,False,4,Heterozygosity (0.0),Genetic Markers,0.009976,0.696801,0.485960,0.984174,0.313399,0.006928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16167,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17519,Methods of self-harm used (0.2),Mental Health,-0.182805,1.464076,0.148225,0.818487,0.829078,0.086988
16168,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17521,Actions taken following self-harm (0.1),Mental Health,-0.020585,0.238343,0.811979,1.000000,0.090455,-0.000000
16169,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17522,Actions taken following self-harm (0.2),Mental Health,0.146455,1.238687,0.219599,0.880943,0.658370,0.055052
16170,Subject-Direction ICA Component 2,5.509794,False,3.201534,False,17525,Ever been offered/sought treatment for anxiety...,Mental Health,-0.023521,1.486123,0.137326,0.802019,0.862249,0.095815


In [14]:
df.to_csv('Famale_ALL_nIDPs_ICA.csv', index=False)

In [15]:
df_passing_FDR = df[df['Passes FDR']==True]

In [16]:
df_passing_FDR

,Modality,Bonferroni Threshold,Passes Bonf.,False Discovery Rate (FDR),Passes FDR,Caterogy ID,Variable Description,Variable Category,Correlation (r),T-Values,P-Values,FDR Adjusted P-Value,-log(P),FDR Adjusted -log(P)
160,Subject-Direction ICA Component 1,5.509794,False,3.524662,True,166,Father's age at death (2.0),Lifestyle,0.058138,3.737560,1.883351e-04,0.034611,3.725069,1.460788
916,Subject-Direction ICA Component 1,5.509794,True,3.524662,True,1131,Pork intake (2.0),Diet,-0.073752,5.217796,1.884126e-07,0.000102,6.724890,3.992428
2401,Subject-Direction ICA Component 1,5.509794,True,3.524662,True,2778,Alcohol intake frequency. (0.0),Alcohol,-0.078131,5.551083,2.985040e-08,0.000025,7.525050,4.595040
2403,Subject-Direction ICA Component 1,5.509794,True,3.524662,True,2780,Alcohol intake frequency. (2.0),Alcohol,-0.076838,5.442281,5.511884e-08,0.000045,7.258700,4.350966
2405,Subject-Direction ICA Component 1,5.509794,False,3.524662,True,2782,Average weekly red wine intake (0.0),Alcohol,-0.064091,3.822212,1.345326e-04,0.030643,3.871172,1.513667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14784,Subject-Direction ICA Component 2,5.509794,False,3.201534,True,16110,Operative procedures - OPCS4 (A843 - A84.3 Ner...,Medical History,0.049424,3.505751,4.593018e-04,0.037705,3.337902,1.423604
15239,Subject-Direction ICA Component 2,5.509794,False,3.201534,True,16567,Operative procedures - OPCS4 (M382 - M38.2 Cys...,Medical History,0.058295,4.136955,3.577026e-05,0.003831,4.446478,2.416691
15574,Subject-Direction ICA Component 2,5.509794,False,3.201534,True,16902,Operative procedures - OPCS4 (T811 - T81.1 Per...,Medical History,0.054494,3.866377,1.118610e-04,0.010898,3.951321,1.962666
15964,Subject-Direction ICA Component 2,5.509794,False,3.201534,True,17297,Frequency of tiredness / lethargy in last 2 we...,Mental Health,-0.248582,3.592921,4.135227e-04,0.034831,3.383501,1.458038


In [17]:
df_passing_FDR.to_csv('Famale_ALL_nIDPs_passing_FDR_ICA.csv', index=False)